In [1]:
import os
from time import sleep
from datetime import date
from datetime import datetime
from datetime import timedelta
import sqlite3

In [2]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys

url = "https://comtrade.un.org/data/ais"

In [3]:
###### FORM ISIAN ######

# isian reporter, ketik nama negara (bisa isi lebih dari satu, default = World)
reporter = ["Indonesia"]

# isian vessel type, 
# All, Bulk, Container_gcargo, Foodstuff, LPG/LNG, Oil/chemicals, Total, Vehicles 
# (bisa isi lebih dari satu, default = Total)
vessel = ["All"]

# isian flows, 
# All, Exports, Imports
# (bisa isi lebih dari satu, default = All)
flows = ["All"]

# isian date, format yyyy-mm-dd, default = 30 hari yg lalu dari tanggal sekarang
date_to = date.today()
date_from = date_from = datetime.now() - timedelta(30)
date_from = date_from.date()
date_from = date_from.strftime("%Y-%m-%d")
date_to = date_to.strftime("%Y-%m-%d")

# date_to = 
# date_from = 

In [4]:
# from selenium.webdriver.chrome.options import Options
# options = Options()
# options.binary_location = wd + "/chromeDriver/GoogleChromePortable/App/Chrome-bin/chrome.exe"

# driver = webdriver.Chrome(wd+"/chromeDriver/chromedriver.exe", options=options)
# driver.close()

In [7]:
# membuat fungsi rekap otomatis simpan ke database

def scrap_ais(reporter, vessel, flows, date_from, date_to):
    driver = webdriver.Chrome("./chromedriver")

    driver.get(url)
    while True:
        elem = driver.find_element_by_id("wrap").find_element_by_class_name("ct-form")
        temp = elem.find_elements_by_xpath("//li[@class='select2-search-choice']/a")
        if len(temp) == 3:
            break

    for t in temp:
        t.click()
        sleep(0.1)

    ####### REPORTER #######

    # dapatkan elemen reporters
    elem_reporters = elem.find_elements_by_class_name("row")[1].find_elements_by_class_name("col-md-3")[0].find_element_by_class_name("select2-choices")

    # isian reporter ganti dengan isian yang sudah ditentukan
    elem_reporters.click()
    for rep in reporter:
        elem_reporters.find_element_by_class_name("select2-search-field").find_element_by_tag_name("input").send_keys(rep)
        elem_reporters.find_element_by_class_name("select2-search-field").find_element_by_tag_name("input").send_keys(Keys.ENTER)
        sleep(0.1)

    ####### VESSEL TYPE #######

    # dapatkan elemen reporters
    elem_vessel = elem.find_elements_by_class_name("row")[1].find_elements_by_class_name("col-md-3")[1].find_element_by_class_name("select2-choices")

    # isian reporter ganti dengan isian yang sudah ditentukan
    elem_vessel.click()
    for rep in vessel:
        elem_vessel.find_element_by_class_name("select2-search-field").find_element_by_tag_name("input").send_keys(rep)
        elem_vessel.find_element_by_class_name("select2-search-field").find_element_by_tag_name("input").send_keys(Keys.ENTER)
        sleep(0.1)

    ####### FLOWS ########

    # dapatkan elemen reporters
    elem_flows = elem.find_elements_by_class_name("row")[1].find_elements_by_class_name("col-md-3")[2].find_element_by_class_name("select2-choices")

    # isian reporter ganti dengan isian yang sudah ditentukan
    elem_flows.click()
    for rep in flows:
        elem_flows.find_element_by_class_name("select2-search-field").find_element_by_tag_name("input").send_keys(rep)
        elem_flows.find_element_by_class_name("select2-search-field").find_element_by_tag_name("input").send_keys(Keys.ENTER)
        sleep(0.1)

    ###### DATE ######
    # dapatkan elemen reporters
    elem_date = elem.find_elements_by_class_name("row")[1].find_elements_by_class_name("col-md-3")[3]
    elem_date.find_element_by_id("dateFrom").clear()
    elem_date.find_element_by_id("dateFrom").send_keys(date_from)
    sleep(0.1)

    elem_date.find_element_by_id("dateTo").clear()
    elem_date.find_element_by_id("dateTo").send_keys(date_to)
    sleep(0.1)

    elem.click()

    # ###### PREVIEW DATA ######
    elem.find_element_by_id("preview").click()

    # pastikan loadingnya selesai
    while True:
        sleep(1)
        temp = driver.find_element_by_id("ct-waiting")
        if (temp.get_attribute("style") == "display: none;"):
            print("load data selesai")
            break

    ###### CONNECT KE DATABASE ######
    import sqlite3

    try:
        sqliteConnection = sqlite3.connect('ais03.db')
        sqlite_create_table_query = '''CREATE TABLE IF NOT EXISTS aistrade ( 
        id INTEGER PRIMARY KEY AUTOINCREMENT, 
        vessel_type VARCHAR(20), 
        flow VARCHAR(10), 
        dates date, 
        num_pc REAL, 
        mtc REAL, 
        dwt REAL, 
        num_pc_ma REAL, 
        mtc_ma REAL, 
        dwt_ma REAL );'''

        cursor = sqliteConnection.cursor()
        print("Successfully Connected to SQLite")
        cursor.execute(sqlite_create_table_query)
        sqliteConnection.commit()
        print("SQLite table created")

    #     cursor.close()

    except sqlite3.Error as error:
        print("Error while creating a sqlite table", error)
    finally:
    #     if (sqliteConnection):
    #         sqliteConnection.close()
    #         print("sqlite connection is closed")
        pass        

    ###### MEMBACA ISI TABEL ######
    # algoritma: 
    # pilih dulu tab tabel
    # cari dulu tabny ada berapa banyak
    # lalu balik ke first
    # perulangan klik next sebanyak (jumlah tab)-1

    # 1. pilih tab tabel
    table_container = driver.find_element_by_id("preview-table-container")

    # 2. cari jumlah tab
    table_container.find_element_by_id("preview-table_last").click()
    tag_a = table_container.find_element_by_id("preview-table_paginate").find_element_by_tag_name("span").find_elements_by_tag_name("a")
    jumlah_tab = int(tag_a[-1].text)

    # 3. balik ke first tab
    table_container.find_element_by_id("preview-table_first").click()



    ###### MEMBACA ISI TABEL ######
    # algoritma: 
    # pilih dulu tab tabel
    # cari dulu tabny ada berapa banyak
    # lalu balik ke first
    # perulangan klik next sebanyak (jumlah tab)-1

    # 1. pilih tab tabel
    table_container = driver.find_element_by_id("preview-table-container")

    # 2. cari jumlah tab
    table_container.find_element_by_id("preview-table_last").click()
    tag_a = table_container.find_element_by_id("preview-table_paginate").find_element_by_tag_name("span").find_elements_by_tag_name("a")
    jumlah_tab = int(tag_a[-1].text)

    # 3. balik ke first tab
    table_container.find_element_by_id("preview-table_first").click()
    
    # 4. perulangan baca tab
    for tab in range(0, jumlah_tab):
    # 5. membaca tiap baris dari setiap tab, lalu colom dari setiap baris
        table_rows = table_container.find_element_by_id("preview-table").find_element_by_tag_name("tbody").find_elements_by_tag_name("tr")
        for row in table_rows:
            temp = []
            cols = row.find_elements_by_tag_name("td")
            for col in cols:
                temp.append(col.text)
            # utk tanggal diambil tanggalnya saja
            temp[3] = temp[3].split("T")[0]

            # hapus array awal karena negaranya pasti Indonesia
            temp.pop(0)

            # masukkan ke database
            sql = '''
            INSERT INTO aistrade
            (vessel_type, flow, dates, num_pc, mtc, dwt, num_pc_ma, mtc_ma, dwt_ma)
            VALUES (?, ?, ?, ?, ?, ?, ?, ?, ?)'''
            cursor.execute(sql, temp)
            sqliteConnection.commit()

        table_container.find_element_by_id("preview-table_next").click()

    cursor.close()
    sqliteConnection.close()

    driver.close()

In [8]:
# rekap utk bulan-04 sd bulan-12 di tahun 2016

for j in range(2021, 2022):
    tahun = j
    for i in range(1,5):
        bulan = i


        # minggu 1
        print("minggu 1 bulan", bulan, "tahun", tahun)
        date_from = str(tahun) + "-" + "{:02d}".format(bulan) + "-" + "01"
        date_to   = str(tahun) + "-" + "{:02d}".format(bulan) + "-" + "07"
        scrap_ais(reporter=reporter, vessel=vessel, flows=flows, date_from=date_from, date_to=date_to)

        # minggu 2
        print("minggu 2 bulan", bulan, "tahun", tahun)
        date_from = str(tahun) + "-" + "{:02d}".format(bulan) + "-" + "08"
        date_to   = str(tahun) + "-" + "{:02d}".format(bulan) + "-" + "14"
        scrap_ais(reporter=reporter, vessel=vessel, flows=flows, date_from=date_from, date_to=date_to)  

        # minggu 3
        print("minggu 3 bulan", bulan, "tahun", tahun)
        date_from = str(tahun) + "-" + "{:02d}".format(bulan) + "-" + "15"
        date_to   = str(tahun) + "-" + "{:02d}".format(bulan) + "-" + "21"
        scrap_ais(reporter=reporter, vessel=vessel, flows=flows, date_from=date_from, date_to=date_to)  

        # minggu 4
        print("minggu 4 bulan", bulan, "tahun", tahun)
        date_from = str(tahun) + "-" + "{:02d}".format(bulan) + "-" + "22"

        if bulan < 12 :
            date_time_str = str(tahun) + "-" + "{:02d}".format(bulan+1) + "-" + "01"
        else:
            date_time_str = str(tahun+1) + "-" + "01" + "-" + "01"

        date_to = datetime.strptime(date_time_str, '%Y-%m-%d')
        date_to = date_to - timedelta(1)
        date_to = date_to.strftime("%Y-%m-%d")

        scrap_ais(reporter=reporter, vessel=vessel, flows=flows, date_from=date_from, date_to=date_to)


minggu 1 bulan 1 tahun 2021
load data selesai
Successfully Connected to SQLite
SQLite table created
minggu 2 bulan 1 tahun 2021
load data selesai
Successfully Connected to SQLite
SQLite table created
minggu 3 bulan 1 tahun 2021
load data selesai
Successfully Connected to SQLite
SQLite table created
minggu 4 bulan 1 tahun 2021
load data selesai
Successfully Connected to SQLite
SQLite table created
minggu 1 bulan 2 tahun 2021
load data selesai
Successfully Connected to SQLite
SQLite table created
minggu 2 bulan 2 tahun 2021
load data selesai
Successfully Connected to SQLite
SQLite table created
minggu 3 bulan 2 tahun 2021
load data selesai
Successfully Connected to SQLite
SQLite table created
minggu 4 bulan 2 tahun 2021
load data selesai
Successfully Connected to SQLite
SQLite table created
minggu 1 bulan 3 tahun 2021
load data selesai
Successfully Connected to SQLite
SQLite table created
minggu 2 bulan 3 tahun 2021
load data selesai
Successfully Connected to SQLite
SQLite table created


IndexError: list index out of range

In [ ]:
# rekap utk bulan-04 sd bulan-12 di tahun 2016


tahun = 2019
for i in range(1,13):
    bulan = i


    # minggu 1
    print("minggu 1 bulan", bulan, "tahun", tahun)
    date_from = str(tahun) + "-" + "{:02d}".format(bulan) + "-" + "01"
    date_to   = str(tahun) + "-" + "{:02d}".format(bulan) + "-" + "07"
    scrap_ais(reporter=reporter, vessel=vessel, flows=flows, date_from=date_from, date_to=date_to)

    # minggu 2
    print("minggu 2 bulan", bulan, "tahun", tahun)
    date_from = str(tahun) + "-" + "{:02d}".format(bulan) + "-" + "08"
    date_to   = str(tahun) + "-" + "{:02d}".format(bulan) + "-" + "14"
    scrap_ais(reporter=reporter, vessel=vessel, flows=flows, date_from=date_from, date_to=date_to)  

    # minggu 3
    print("minggu 3 bulan", bulan, "tahun", tahun)
    date_from = str(tahun) + "-" + "{:02d}".format(bulan) + "-" + "15"
    date_to   = str(tahun) + "-" + "{:02d}".format(bulan) + "-" + "21"
    scrap_ais(reporter=reporter, vessel=vessel, flows=flows, date_from=date_from, date_to=date_to)  

    # minggu 4
    print("minggu 4 bulan", bulan, "tahun", tahun)
    date_from = str(tahun) + "-" + "{:02d}".format(bulan) + "-" + "22"

    if bulan < 12 :
        date_time_str = str(tahun) + "-" + "{:02d}".format(bulan+1) + "-" + "01"
    else:
        date_time_str = str(tahun+1) + "-" + "01" + "-" + "01"

    date_to = datetime.strptime(date_time_str, '%Y-%m-%d')
    date_to = date_to - timedelta(1)
    date_to = date_to.strftime("%Y-%m-%d")

    scrap_ais(reporter=reporter, vessel=vessel, flows=flows, date_from=date_from, date_to=date_to)


In [ ]:
# rekap utk bulan-04 sd bulan-12 di tahun 2016


tahun = 2019
for i in range(9,13):
    bulan = i


    # minggu 1
    print("minggu 1 bulan", bulan, "tahun", tahun)
    date_from = str(tahun) + "-" + "{:02d}".format(bulan) + "-" + "01"
    date_to   = str(tahun) + "-" + "{:02d}".format(bulan) + "-" + "07"
    scrap_ais(reporter=reporter, vessel=vessel, flows=flows, date_from=date_from, date_to=date_to)

    # minggu 2
    print("minggu 2 bulan", bulan, "tahun", tahun)
    date_from = str(tahun) + "-" + "{:02d}".format(bulan) + "-" + "08"
    date_to   = str(tahun) + "-" + "{:02d}".format(bulan) + "-" + "14"
    scrap_ais(reporter=reporter, vessel=vessel, flows=flows, date_from=date_from, date_to=date_to)  

    # minggu 3
    print("minggu 3 bulan", bulan, "tahun", tahun)
    date_from = str(tahun) + "-" + "{:02d}".format(bulan) + "-" + "15"
    date_to   = str(tahun) + "-" + "{:02d}".format(bulan) + "-" + "21"
    scrap_ais(reporter=reporter, vessel=vessel, flows=flows, date_from=date_from, date_to=date_to)  

    # minggu 4
    print("minggu 4 bulan", bulan, "tahun", tahun)
    date_from = str(tahun) + "-" + "{:02d}".format(bulan) + "-" + "22"

    if bulan < 12 :
        date_time_str = str(tahun) + "-" + "{:02d}".format(bulan+1) + "-" + "01"
    else:
        date_time_str = str(tahun+1) + "-" + "01" + "-" + "01"

    date_to = datetime.strptime(date_time_str, '%Y-%m-%d')
    date_to = date_to - timedelta(1)
    date_to = date_to.strftime("%Y-%m-%d")

    scrap_ais(reporter=reporter, vessel=vessel, flows=flows, date_from=date_from, date_to=date_to)


In [ ]:
# rekap utk bulan-04 sd bulan-12 di tahun 2016


tahun = 2020
for i in range(1,7):
    bulan = i


    # minggu 1
    print("minggu 1 bulan", bulan, "tahun", tahun)
    date_from = str(tahun) + "-" + "{:02d}".format(bulan) + "-" + "01"
    date_to   = str(tahun) + "-" + "{:02d}".format(bulan) + "-" + "07"
    scrap_ais(reporter=reporter, vessel=vessel, flows=flows, date_from=date_from, date_to=date_to)

    # minggu 2
    print("minggu 2 bulan", bulan, "tahun", tahun)
    date_from = str(tahun) + "-" + "{:02d}".format(bulan) + "-" + "08"
    date_to   = str(tahun) + "-" + "{:02d}".format(bulan) + "-" + "14"
    scrap_ais(reporter=reporter, vessel=vessel, flows=flows, date_from=date_from, date_to=date_to)  

    # minggu 3
    print("minggu 3 bulan", bulan, "tahun", tahun)
    date_from = str(tahun) + "-" + "{:02d}".format(bulan) + "-" + "15"
    date_to   = str(tahun) + "-" + "{:02d}".format(bulan) + "-" + "21"
    scrap_ais(reporter=reporter, vessel=vessel, flows=flows, date_from=date_from, date_to=date_to)  

    # minggu 4
    print("minggu 4 bulan", bulan, "tahun", tahun)
    date_from = str(tahun) + "-" + "{:02d}".format(bulan) + "-" + "22"

    if bulan < 12 :
        date_time_str = str(tahun) + "-" + "{:02d}".format(bulan+1) + "-" + "01"
    else:
        date_time_str = str(tahun+1) + "-" + "01" + "-" + "01"

    date_to = datetime.strptime(date_time_str, '%Y-%m-%d')
    date_to = date_to - timedelta(1)
    date_to = date_to.strftime("%Y-%m-%d")

    scrap_ais(reporter=reporter, vessel=vessel, flows=flows, date_from=date_from, date_to=date_to)


In [ ]:
tahun = 2020
for i in range(7,13):
    bulan = i


    # minggu 1
    print("minggu 1 bulan", bulan, "tahun", tahun)
    date_from = str(tahun) + "-" + "{:02d}".format(bulan) + "-" + "01"
    date_to   = str(tahun) + "-" + "{:02d}".format(bulan) + "-" + "07"
    scrap_ais(reporter=reporter, vessel=vessel, flows=flows, date_from=date_from, date_to=date_to)

    # minggu 2
    print("minggu 2 bulan", bulan, "tahun", tahun)
    date_from = str(tahun) + "-" + "{:02d}".format(bulan) + "-" + "08"
    date_to   = str(tahun) + "-" + "{:02d}".format(bulan) + "-" + "14"
    scrap_ais(reporter=reporter, vessel=vessel, flows=flows, date_from=date_from, date_to=date_to)  

    # minggu 3
    print("minggu 3 bulan", bulan, "tahun", tahun)
    date_from = str(tahun) + "-" + "{:02d}".format(bulan) + "-" + "15"
    date_to   = str(tahun) + "-" + "{:02d}".format(bulan) + "-" + "21"
    scrap_ais(reporter=reporter, vessel=vessel, flows=flows, date_from=date_from, date_to=date_to)  

    # minggu 4
    print("minggu 4 bulan", bulan, "tahun", tahun)
    date_from = str(tahun) + "-" + "{:02d}".format(bulan) + "-" + "22"

    if bulan < 12 :
        date_time_str = str(tahun) + "-" + "{:02d}".format(bulan+1) + "-" + "01"
    else:
        date_time_str = str(tahun+1) + "-" + "01" + "-" + "01"

    date_to = datetime.strptime(date_time_str, '%Y-%m-%d')
    date_to = date_to - timedelta(1)
    date_to = date_to.strftime("%Y-%m-%d")

    scrap_ais(reporter=reporter, vessel=vessel, flows=flows, date_from=date_from, date_to=date_to)


In [ ]:
tahun = 2021
for i in range(1,3):
    bulan = i


    # minggu 1
    print("minggu 1 bulan", bulan, "tahun", tahun)
    date_from = str(tahun) + "-" + "{:02d}".format(bulan) + "-" + "01"
    date_to   = str(tahun) + "-" + "{:02d}".format(bulan) + "-" + "07"
    scrap_ais(reporter=reporter, vessel=vessel, flows=flows, date_from=date_from, date_to=date_to)

    # minggu 2
    print("minggu 2 bulan", bulan, "tahun", tahun)
    date_from = str(tahun) + "-" + "{:02d}".format(bulan) + "-" + "08"
    date_to   = str(tahun) + "-" + "{:02d}".format(bulan) + "-" + "14"
    scrap_ais(reporter=reporter, vessel=vessel, flows=flows, date_from=date_from, date_to=date_to)  

    # minggu 3
    print("minggu 3 bulan", bulan, "tahun", tahun)
    date_from = str(tahun) + "-" + "{:02d}".format(bulan) + "-" + "15"
    date_to   = str(tahun) + "-" + "{:02d}".format(bulan) + "-" + "21"
    scrap_ais(reporter=reporter, vessel=vessel, flows=flows, date_from=date_from, date_to=date_to)  

    # minggu 4
    print("minggu 4 bulan", bulan, "tahun", tahun)
    date_from = str(tahun) + "-" + "{:02d}".format(bulan) + "-" + "22"

    if bulan < 12 :
        date_time_str = str(tahun) + "-" + "{:02d}".format(bulan+1) + "-" + "01"
    else:
        date_time_str = str(tahun+1) + "-" + "01" + "-" + "01"

    date_to = datetime.strptime(date_time_str, '%Y-%m-%d')
    date_to = date_to - timedelta(1)
    date_to = date_to.strftime("%Y-%m-%d")

    scrap_ais(reporter=reporter, vessel=vessel, flows=flows, date_from=date_from, date_to=date_to)
